<a href="https://colab.research.google.com/github/Vishwabvs/COVID-19-Prediction-Using-X-Ray-Images/blob/master/Covid19_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle 

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chaitanyabvs","key":"f4145b5fd94e48c0767a83abd072d774"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d prashant268/chest-xray-covid19-pneumonia

100% 1.65G/1.65G [00:34<00:00, 55.9MB/s]
100% 1.65G/1.65G [00:34<00:00, 50.9MB/s]


In [5]:
from zipfile import ZipFile
file_name = 'chest-xray-covid19-pneumonia.zip'

with ZipFile(file_name, 'r') as zip:
    zip.extractall()
    print('done')

done


## **Importing Libraries**

In [41]:
import numpy as np
import pandas as pd
import os 
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten, MaxPool2D
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator

In [42]:
IMG_SIZE = 224
train_datadir = '/content/Data/train'
test_datadir = '/content/Data/test'

In [43]:
vgg = VGG16(input_shape = [IMG_SIZE, IMG_SIZE, 3], weights = 'imagenet', include_top = False)

In [44]:
for layer in vgg.layers:
    layer.trainable = False

In [45]:
num_classes = 3
X = Flatten()(vgg.output)
prediction = Dense(num_classes, activation = 'softmax')(X)

In [46]:
model = Model(inputs = vgg.input, outputs = prediction)
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [47]:
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [48]:
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)
train_set = train_datagen.flow_from_directory(train_datadir, target_size = (IMG_SIZE, IMG_SIZE), 
                                              batch_size = 32, class_mode = 'categorical', subset = 'training')
val_set = train_datagen.flow_from_directory(train_datadir, target_size = (IMG_SIZE, IMG_SIZE),
                                            batch_size = 32, class_mode = 'categorical', subset = 'validation')

Found 4116 images belonging to 3 classes.
Found 1028 images belonging to 3 classes.


In [49]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(test_datadir, target_size = (IMG_SIZE, IMG_SIZE),
                                            batch_size = 32, class_mode = 'categorical')

Found 1288 images belonging to 3 classes.


In [50]:
r = model.fit_generator(train_set, steps_per_epoch = len(train_set), epochs = 10, validation_data = val_set, validation_steps = len(val_set))

Epoch 1/10
129/129 [==============================] - 70s 543ms/step - loss: 0.3264 - accuracy: 0.8895 - val_loss: 0.1147 - val_accuracy: 0.9630
Epoch 2/10
129/129 [==============================] - 70s 542ms/step - loss: 0.0984 - accuracy: 0.9667 - val_loss: 0.0963 - val_accuracy: 0.9698
Epoch 3/10
129/129 [==============================] - 70s 543ms/step - loss: 0.0895 - accuracy: 0.9730 - val_loss: 0.1507 - val_accuracy: 0.9455
Epoch 4/10
129/129 [==============================] - 70s 544ms/step - loss: 0.0642 - accuracy: 0.9779 - val_loss: 0.1092 - val_accuracy: 0.9621
Epoch 5/10
129/129 [==============================] - 70s 540ms/step - loss: 0.0681 - accuracy: 0.9769 - val_loss: 0.0798 - val_accuracy: 0.9757
Epoch 6/10
129/129 [==============================] - 70s 542ms/step - loss: 0.0391 - accuracy: 0.9903 - val_loss: 0.0845 - val_accuracy: 0.9698
Epoch 7/10
129/129 [==============================] - 70s 542ms/step - loss: 0.0307 - accuracy: 0.9913 - val_loss: 0.0779 - val_ac

In [51]:
predict = model.predict_generator(test_set)

In [29]:
model.evaluate(x = test_set)

41/41 [==============================] - 17s 415ms/step - loss: 0.0896 - accuracy: 0.9635


[0.0896308571100235, 0.9635093212127686]

In [58]:
import cv2
outputs = ['COVID19', 'NORMAL', 'PNEUMONIA']

def prepare(filepath):
    IMG_SIZE = 224
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

prediction = model.predict([prepare('corona2.jpeg')])     #some random covid positive xray
print(outputs[int(prediction[0][0])])

COVID19
